In [1]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

In [39]:
# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import os
import requests
import tarfile
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np

import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.distribute_eia923 as distribute_eia923

# Specify the year for analysis

In [3]:
year = 2020

# 1. Download data

 - Downloads the pre-cleaned PUDL versions of EIA-923, EIA-860, and EPA CEMS data  
 - Downloads EPA eGRID data  
 - Downloads EIA-930 data  
 - Downloads the EPA Power Sector Data Crosswalk

TODO
- [x] The code for downloading the files could probably be made into functions
- [ ] Investigate other packages besides `requests` that would download these files faster

In [4]:
############### PUDL Database ######################

load_data.download_pudl_data(zenodo_url = 'https://zenodo.org/record/5701406/files/pudl-v0.5.0-2021-11-14.tgz')

################# eGRID data #########################

# the 2019 and 2020 data appear to be hosted on different urls
egrid_files_to_download = ['https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx', 
                           'https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx']

load_data.download_egrid_files(egrid_files_to_download)

############# EIA-930 data #####################

load_data.download_eia930_data(years_to_download=[year])

########## Power Sector Data Crosswalk #############
# NOTE: Check for new releases at https://github.com/USEPA/camd-eia-crosswalk

load_data.download_epa_psdc(psdc_url='https://github.com/USEPA/camd-eia-crosswalk/releases/download/v0.2.1/epa_eia_crosswalk.csv')


PUDL data already downloaded
egrid2019_data.xlsx already downloaded
egrid2020_data.xlsx already downloaded
2020_Jan_Jun data already downloaded
2020_Jul_Dec data already downloaded
epa_eia_crosswalk.csv already downloaded


# Load emissions data reported to CEMS
There are three broad categories of plants based on their CAMD reporting status:
1. Units that report to CAMD year-round (for these plants, emissions data is used directly from CEMS)
2. Units that only report to CAMD during the ozone season (May-Sept) (for these units, non-ozone season data is taken from EIA 923)
3. Units that do not report to CAMD (generally fossil units < 25MW and non-fossil generators)

There are also certain plants that report to CAMD but do not procuce electricty for the grid, and need to be removed from the CEMS data:
- Non grid connected plants
- Steam-only plants


In [ ]:
# NOTE: all of the functions in this section could be run by calling clean_cems()
#cems = data_cleaning.clean_cems(year)

In [40]:
# load the CEMS data
cems = load_data.load_cems_data(year)

# remove non-grid connected plants
cems = data_cleaning.remove_non_grid_connected_plants(cems)

# remove plants that only report steam generation and no electrical generation
cems = data_cleaning.remove_heating_only_plants(cems)

# add a report date
cems = data_cleaning.add_report_date(cems)

# identify cems reporting status
cems = data_cleaning.determine_cems_reporting_status(cems)

# TODO: identify and remove any hourly values that appear to be outliers

In [42]:
# fill in missing hourly emissions data using the fuel type and heat input
# TODO: test geothermal emissions filling
# TODO: fill based on weighted average EF from 923?
cems = data_cleaning.fill_cems_missing_co2(cems, year)

In [12]:
# identify any remaining missing values
# TODO: Try to identify fuel types
# NOTE: plant 880109 appears to be a paper mill in Ohio, so should maybe be added to non-grid connected

units_with_no_fuel_type = list(cems[cems['co2_mass_tons'].isnull()]['cems_id'].unique())
print(f"Unable to find fuel types for the following plants_units: {units_with_no_fuel_type}")

cems[cems['co2_mass_tons'].isnull()]

Unable to find fuel types for the following plants_units: ['1004_CTG1', '880109_B001']


,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,co2_mass_measurement_code,plant_id_epa,unit_id_epa,report_date,cems_reporting_category
10905359,1004,CTG1,1004_CTG1,2020-04-10 04:00:00+00:00,0.03,0.0,0.0,NaN,0.000900,NaN,Measured,1004,90673,2020-04-01,full_year
11497174,1004,CTG1,1004_CTG1,2020-11-02 03:00:00+00:00,0.03,0.0,0.0,NaN,0.000900,NaN,Measured,1004,90673,2020-11-01,full_year
23945496,880109,B001,880109_B001,2020-05-01 05:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-05-01,partial_year
23945497,880109,B001,880109_B001,2020-05-01 06:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-05-01,partial_year
23945498,880109,B001,880109_B001,2020-05-01 07:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-05-01,partial_year
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24290251,880109,B001,880109_B001,2020-10-01 00:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-09-01,partial_year
24290252,880109,B001,880109_B001,2020-10-01 01:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-09-01,partial_year
24290253,880109,B001,880109_B001,2020-10-01 02:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-09-01,partial_year
24290254,880109,B001,880109_B001,2020-10-01 03:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,NaN,880109,91300,2020-09-01,partial_year


In [13]:
# For now, lets drop these from the data
cems = cems[~cems['cems_id'].isin(units_with_no_fuel_type)]

In [14]:
# remove any observations from cems where zero operation is reported for an entire month
# although this data could be considered to be accurately reported, let's remove it so that we can double check against the eia data
# TODO: check if any of these observations are from geothermal generators
cems = data_cleaning.remove_cems_with_zero_monthly_emissions(cems)

removing 7002048 observations from cems


In [ ]:
# add information about the balancing authority and fuel type

## Explore outlier detection
We need to come up with a method that filters out observations that are significantly higher than normal operation. The challenge is that some plants only operate a handful of hours each year, so their operation looks spikey, which would be identified as an outlier using typical detection methods. 

In [15]:
max = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].max()
mean = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].mean()
stdev = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].std()

In [16]:
max[max > mean + (3 * stdev)]

cems_id
1012_2         1427.199951
10298_CG803    2463.600098
1040_1          620.200012
1040_2         1201.500000
10823_S42       290.100006
                  ...     
963_32         1203.800049
976_123        1756.500000
976_4          2869.600098
983_1          2435.899902
983_4          2586.800049
Name: heat_content_mmbtu, Length: 271, dtype: float32

In [17]:
px.line(cems[cems['cems_id'] == '10298_CG803'], x='operating_datetime_utc', y='heat_content_mmbtu')

In [20]:
px.line(cems[cems['cems_id'] == '1012_2'], x='operating_datetime_utc', y='heat_content_mmbtu')

# 2. Get monthly data for all plants/units where data is missing from CEMS
We have now identified all plants that report the full year to CEMS, and all plants that report a partial year. We will now use the EIA-923 data to fill in the missing pieces.

1. Load EIA-923 data, and standardize heat input and generation data across the tables
2. Identify all plants/months for which we do not have CEMS data

We need to be able to match the EIA data to the CEMS data based on units so we know which data will be used to fill the missing data


In [31]:
# Distribute net generation and heat input data reported by the three different EIA-923 tables
# NOTE: this code was copied and modified from `pudl.analysis.allocate_net_gen`
# NOTE: this code allocates net generation based on the proportion of net generation reported, rather than by nameplate capacity (which eGRID does)
# NOTE: the code was modified to perform the allocation on a monthly basis, rather than an annual basis

# HIGH PRIORITIES
# DONE: remove non-grid connected plants from this dataframe
# DONE: Denormalize data by balancing authority/state. BA assignment from EIA-860
# TODO: look into whether net_gen from generation table should be preserved if available

# FUEL ASSIGNMENT
# DONE: calculate total emissions from gf based on fuel and heat input and distribute in addition to net generation and fuel consumed
# TODO: when aggregating back to generator records, keep the fuel type that accounted for most heat input
# TODO: denormalize data by fuel type both primary fuel type by generator, and primary fuel by plant (assuming that's how reported to ISOs)
# primary fuel type is currently assigned based on the annual primary fuel type. This should be changed to assign base on monthly fuel type

# NEXT PRIORITIES
# TODO: allocate heat input data from boiler_fuel_eia923() See: https://github.com/catalyst-cooperative/pudl/pull/1096
# TODO: Also distribute heat input for electricity consumption

# LOWER PRIORITIES
# TODO: fix allocation of net generation when reported net generation is negative?
# TODO: investigate generators for which frac column is not adding to 1.0
# TODO: add nuclear generation?

gen_fuel_allocated = distribute_eia923.allocate_gen_fuel_by_gen(year=year)

# flag any generator-months for which we already have cems data
gen_fuel_allocated = data_cleaning.identify_emissions_data_source(cems, gen_fuel_allocated)

gen_fuel_allocated.sample(10)

c:\Users\Greg\GitHub\hourly-egrid\notebooks\../../hourly-egrid\src\distribute_eia923.py:848: UserWarning:

Ooopsies. You got 249 records where the 'frac' column isn't adding up to 1 for each 'IDX_PM_FUEL' group. Check 'calc_allocation_fraction()'
        plant_id_eia prime_mover_code energy_source_code report_date  frac  \
3413             377               CA                 NG  2020-04-01   2.0   
3414             377               CT                 NG  2020-04-01   2.0   
3441             377               CA                 NG  2020-11-01   2.0   
3442             377               CT                 NG  2020-11-01   2.0   
3445             377               CA                 NG  2020-12-01   2.0   
...              ...              ...                ...         ...   ...   
104483         58207               CA                 NG  2020-03-01   4.0   
104487         58207               CA                 NG  2020-04-01   4.0   
104495         58207               CA              

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons,balancing_authority_code_eia,state,energy_source_code_1,data_source
102908,8083,7,2020-05-01,7128.000000,0.000000,0.000000,None,HI,DFO,eia_only
205385,58266,6,2020-09-01,3985.160083,34295.250000,2004.214410,PGE,OR,NG,eia_only
253914,61746,BCSD,2020-10-01,6780.272000,59483.000000,0.000000,SWPP,SD,WND,eia_only
143006,54984,UNT2,2020-09-01,2238.854000,34232.000000,2173.732000,DUK,NC,LFG,eia_only
187961,57237,S022D,2020-01-01,22.246250,195.250000,0.000000,CISO,CA,SUN,eia_only
239107,60709,GRND,2020-11-01,285.680000,2506.000000,0.000000,PJM,NJ,SUN,eia_only
82238,6393,2,2020-11-01,257.936667,2263.000000,0.000000,BPAT,WY,WAT,eia_only
200810,58022,NHS3,2020-10-01,5100.680333,44748.333333,NaN,IPCO,OR,GEO,eia_only
150662,55279,CTG2,2020-12-01,3601.352237,37471.640582,2189.842676,PJM,IL,NG,cems
29734,1488,DR4,2020-05-01,574.208862,5037.538462,0.000000,ISNE,ME,WAT,eia_only


In [16]:
# create a separate dataframe containing only the generators for which we do not have CEMS data
monthly_eia_data_to_distribute = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'eia_only') & ~(gen_fuel_allocated['fuel_consumed_mmbtu'].isna())]

In [ ]:
# TODO: Compare the fuel input from CEMS to the input from EIA to see if they are close
# in general, we will trust the CEMS data over the EIA data unless there are significant differences
# may need to aggregate to plant level since there is not a 1:1 match between units and generators

# for plants where there is data reported in cems, see how off it is from data reported in eia
cems_plant_monthly = cems.groupby(['plant_id_eia','report_date']).sum()[['heat_content_mmbtu']].reset_index()
gf_plant_monthly = gen_fuel_allocated.groupby(['plant_id_eia','report_date']).sum().reset_index()
gf_plant_monthly = gf_plant_monthly.merge(cems_plant_monthly, how='left', on=['plant_id_eia','report_date'])
gf_plant_monthly = gf_plant_monthly[gf_plant_monthly['heat_content_mmbtu'].notnull()]
gf_plant_monthly['pct_diff'] = (gf_plant_monthly['heat_content_mmbtu'] - gf_plant_monthly['fuel_consumed_mmbtu']) / gf_plant_monthly['fuel_consumed_mmbtu']

# identify where there are plants that report 0 heat input to cems but have data in eia_923
gf_plant_monthly[(gf_plant_monthly['heat_content_mmbtu'] == 0) & (gf_plant_monthly['fuel_consumed_mmbtu'] > 0)]

In [ ]:
# TODO: Add fuel categories (matching those used by EIA-930) to gen_fuel_allocated so that we can roll it up to the BA-fuel level.


In [ ]:
# TODO: Check the total heat input from monthly_eia_data_to_distribute and cems against the total heat input reported in eGRID2020 to make sure they match

## Calculate CEMS net generation
Now that we have accurate net generation data from EIA, we can use this to calculate a net generation ratio to convert the CEMS gross generation to hourly net generation

In [ ]:
# NOTE: This is old code and hasn't been updated yet

generators = pd.read_sql('generation_eia923', pudl_engine)

# Some columns (eg, date) do not make sense to aggregate
aggregate_cols = ['plant_id_eia', 'operating_datetime_utc','co2_mass_tons', 'heat_content_mmbtu', 'gross_generation_mwh', 'gross_load_mw']

# calculate parastic loss factors
gtn_ratios, gtn_fill_values = gross_to_net_generation.gross_to_net_ratios(cems, generators, plants_entity_eia)

print(' aggregating data to plant level')
# aggregate to plant level
# drop columns that will not be aggregated
cems_gross = cems[aggregate_cols]
cems_gross = cems_gross.groupby(['plant_id_eia', 'operating_datetime_utc']).sum().reset_index()

print(' adding report dates')
# add report_date column
cems_gross = data_cleaning.add_report_date(cems_gross, plants_entity_eia)

print(' calculating net generation')
# convert gross load to net load
cems_gross = cems_gross.merge(gtn_ratios[['plant_id_eia', 'report_date', 'gtn_ratio']], how='left', on=['plant_id_eia', 'report_date'])

#fillna with average of non-na values for same plant
cems_gross = cems_gross.merge(gtn_fill_values, how='left', on=['plant_id_eia'])
cems_gross['gtn_ratio'] = cems_gross['gtn_ratio'].fillna(cems_gross['gtn_fill'])
cems_gross = cems_gross.drop(columns=['gtn_fill'])
# any other values that are still missing should be filled with 1
cems_gross['gtn_ratio'] = cems_gross['gtn_ratio'].fillna(1)

#calculate net generation
cems_gross['net_generation_mwh'] = cems_gross['gross_generation_mwh'] * cems_gross['gtn_ratio']

# need to figure out if aggregating at the plant level is ok


In [ ]:
# For what fraction of rows are we using default of 1 as gross-to-net ratio? 
sum(cems_gross["gtn_ratio"] == 1)/len(cems_gross["gtn_ratio"])

In [ ]:
# merge net generation into the main cems dataframe

# These columns should be the same across units within plant, so get first row of each
# TODO: we don't use these columns currently. if that continues to be true, we could drop them.
non_aggregated_cols = ['plant_id_eia', 'plant_id_epa', 'operating_time_hours', 'co2_mass_measurement_code','facility_id','operating_datetime_utc']
plant_cems = cems.groupby(['plant_id_eia', 'operating_datetime_utc']).head(1)

cems = cems_gross.merge(plant_cems[non_aggregated_cols], how='left', on=[
                                    'plant_id_eia', 'operating_datetime_utc'])

# NOTE: the BA data has not yet been merged in
#cems.groupby("balancing_authority_code_eia").sum().head()

# Adjust emissions
We next need to make certain adjustments to the data:
 - [ ] Calculate emissions for Geothermal plants
 - [ ] Adjust heat input/emissions from CHP plants by proportion used for electric generation
 - EPA adjusts biomass emissions, but not sure if we want to do that. Need to look into it more

# Assign monthly data to hourly profile
We now, in theory, have complete data on national-level heat input, net generation, and emissions, from a combination of two sources:
    1. hourly data from CEMS
    2. momthly data for generators that don't report to CEMS

For the second category of monthly data, we need to figure out how to allocate the monthly level data to each hour. 

# Old Code

In [ ]:
# load the generation fuel data for the year
generation_fuel_eia923 = load_data.load_pudl_table(f"SELECT * FROM generation_fuel_eia923 WHERE report_date >= '{year}-01-01' AND report_date <= '{year}-12-01'")

# remove non-grid connected plants
generation_fuel_eia923 = data_cleaning.remove_non_grid_connected_plants(generation_fuel_eia923, year)

generation_fuel_eia923

In [ ]:
# categorize generators by broad fuel categories (clean, geothermal, biofuel, fossil)
clean_fuels = ['SUN','MWH','WND', 'WAT','WH','PUR','NUC']
fossil_fuels = ['NG', 'DFO','OG','WDS','BIT','OTH','PC','SUB', 'LIG','KER', 'RC', 'WO','RFO', 'WC', 'SGC', 'SGP', 'PG', 'JF','BFG']
bio_fuels = ['AB','BG','BLQ','DG','LFG','MSB','MSW','OBG','OBL','OBS','SLW','WDL','WDS']
geo_fuel = ['GEO']
# TODO: Figure out what to do with MSW

# identify and remove all plants that are non-emitting
generation_fuel_eia923 = generation_fuel_eia923[~generation_fuel_eia923['energy_source_code'].isin(clean_fuels)]

In [ ]:
# identify any plants that did not generate any electricity, and remove them from the data
gf923_annual = generation_fuel_eia923.groupby(['plant_id_eia']).sum()
steam_only_plants = list(gf923_annual[(gf923_annual['fuel_consumed_for_electricity_mmbtu'] == 0) & (gf923_annual['fuel_consumed_mmbtu'] > 0)].index)
generation_fuel_eia923 = generation_fuel_eia923[~generation_fuel_eia923['plant_id_eia'].isin(steam_only_plants)]

In [ ]:
generation_fuel_eia923

In [ ]:
generation_eia923 = load_data.load_pudl_table(f"SELECT * FROM generation_fuel_eia923 WHERE report_date >= '{year}-01-01' AND report_date <= '{year}-12-01'")

In [ ]:
# data cleaning

In [ ]:
# calculate emissions based on fuel type
# get emission factors
emission_factors = load_data.load_emission_factors(year)[['energy_source_code', 'co2_tons_per_mmbtu']]
# add emission factor to missing df
generation_fuel_eia923 = generation_fuel_eia923.merge(emission_factors, how='left', on='energy_source_code')
# calculate missing co2 data
generation_fuel_eia923['co2_mass_tons'] = generation_fuel_eia923['fuel_consumed_mmbtu'] * generation_fuel_eia923['co2_tons_per_mmbtu']
generation_fuel_eia923


In [ ]:
generation_fuel_eia923

In [ ]:
# need to calculate emissions for geothermal plants
# need to find a data source that reports this
generation_fuel_eia923[generation_fuel_eia923['energy_source_code'] == 'GEO']

In [ ]:
generation_fuel_eia923[generation_fuel_eia923['co2_tons_per_mmbtu'].isna()]['energy_source_code'].unique()

In [ ]:
# calculate what percent of heat input is covered by CEMS
cems['heat_content_mmbtu'].sum() / generation_fuel_eia923['fuel_consumed_mmbtu'].sum()

In [ ]:
# calculate what percent of emissions is covered by CEMS
cems['co2_mass_tons'].sum() / generation_fuel_eia923['co2_mass_tons'].sum()

# Aggregate data to Balancing Authorities
The output should be a pandas dataframe with a datetimeindex for each hour of the year (in UTC) and each column is a different BA code. The data will represent the average emission rate.

To compare to EIA-930 and annual eGRID, we will also want to compare net generation and emissions.

In [ ]:
# identify which BA each plant is in
plant_ba = plants_entity_eia[['plant_id_eia','balancing_authority_code_eia']]

# merge the ba code into the CEMS data
cems = cems.merge(plant_ba, how='left', on='plant_id_eia')
cems.head(5)

In [ ]:
# Output CEMS data after data processing steps above. 
# Note: this is a big file. If needed could switch to better file format, eg hdf

cems.to_csv(f"../data/output/cems{year}.csv")

In [ ]:
hourly_emissions = cems.groupby(['balancing_authority_code_eia','operating_datetime_utc']).sum()[['co2_mass_tons','net_generation_mwh']]
# divide total emissions by total generation to get the emission factor
hourly_emissions['ef_tons_co2_per_mwh'] = hourly_emissions['co2_mass_tons'] / hourly_emissions['net_generation_mwh']

# pivot the data
hourly_emission_rate = hourly_emissions.reset_index().pivot(index='operating_datetime_utc', columns='balancing_authority_code_eia', values='ef_tons_co2_per_mwh')
hourly_emission_rate

In [ ]:
# Net generation and emission 

hourly_net_generation = hourly_emissions.reset_index().pivot(index='operating_datetime_utc', columns='balancing_authority_code_eia', values='net_generation_mwh')
hourly_net_emissions = hourly_emissions.reset_index().pivot(index='operating_datetime_utc', columns='balancing_authority_code_eia', values='co2_mass_tons')

# 4. Output data 

Save data to a CSV so we can separate generation of and analysis of hourly e-grid numbers

In [ ]:
if not os.path.exists('../data/output'):
    os.makedirs('../data/output')
    
hourly_emission_rate.to_csv(f"../data/output/hourly_emission_rate{year}.csv")
hourly_net_generation.to_csv(f"../data/output/hourly_net_generation{year}.csv")
hourly_net_emissions.to_csv(f"../data/output/hourly_net_emission{year}.csv")